In [1]:
import numpy as np
import pickle
import statistics

from xgboost import XGBClassifier
from sympy.core.numbers import igcd
from datetime import datetime

from sklearn import preprocessing
from sklearn import svm
from sklearn import preprocessing
from sklearn.ensemble import IsolationForest


from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.preprocessing import MinMaxScaler,QuantileTransformer,RobustScaler,StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support as score

import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
import os
import time
import random
from scipy import stats

from pyod.models.hbos import HBOS
from pyod.models.cblof import CBLOF
from pyod.models.loci import LOCI
from pyod.models.xgbod import XGBOD
from pyod.models.cof import COF
from pyod.models.loda import LODA
from pyod.models.copod import COPOD
from pyod.models.sod import SOD
from pyod.models.vae import VAE
from pyod.models.lof import LocalOutlierFactor,LOF
from pyod.models.lscp import LSCP
from pyod.models.so_gaal import SO_GAAL
from pyod.models.mo_gaal import MO_GAAL
from pyod.models.iforest import IForest
from pyod.models.ocsvm import OCSVM
from pyod.models.auto_encoder import AutoEncoder

from keras.models import Sequential
from keras import layers
from keras.models import Model, load_model
from keras.layers import Input, Dense, Dropout,LSTM, Conv1D, MaxPooling1D, AveragePooling1D, Flatten,Reshape,UpSampling1D
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from sklearn.manifold import TSNE
os.environ["CUDA_VISIBLE_DEVICES"] = "2"


In [2]:
# Data load
df_0=pd.read_csv("./Datasets/Rp3_3rd_capture/rp3_normal_0.csv")
df_1=pd.read_csv("./Datasets/Rp3_3rd_capture/rp3_normal_1.csv")
df_2=pd.read_csv("./Datasets/Rp3_3rd_capture/rp3_normal_2.csv")
df_3=pd.read_csv("./Datasets/Rp3_3rd_capture/rp3_normal_3.csv")
df_4=pd.read_csv("./Datasets/Rp3_3rd_capture/rp3_normal_4.csv")
df_5=pd.read_csv("./Datasets/Rp3_3rd_capture/rp3_normal_5.csv")
df_6=pd.read_csv("./Datasets/Rp3_3rd_capture/rp3_normal_6.csv")
df_mix=pd.read_csv("./Datasets/Rp3_3rd_capture/rp3_normal_all.csv")

In [3]:
# Filter vectors according to the timestamps

df_0 = df_0.loc[(df_0['timestamp'] >= 1615419435000) & (df_0['timestamp'] <= 1615451520000)]
df_1 = df_1.loc[(df_1['timestamp'] >= 1615925651000) & (df_1['timestamp'] <= 1615929569000)]
df_2 = df_2.loc[(df_2['timestamp'] >= 1615930017000) & (df_2['timestamp'] <= 1615933673000)]
df_3 = df_3.loc[(df_3['timestamp'] >= 1615969307000) & (df_3['timestamp'] <= 1615972800000)]
df_4 = df_4.loc[(df_4['timestamp'] >= 1615974874000) & (df_4['timestamp'] <= 1615978500000)]
df_5 = df_5.loc[(df_5['timestamp'] >= 1615978899000) & (df_5['timestamp'] <= 1615982700000)]
df_6 = df_6.loc[(df_6['timestamp'] >= 1615989015000) & (df_6['timestamp'] <= 1615992622000)]
df_mix = df_mix.loc[(df_mix['timestamp'] >= 1615996500000) & (df_mix['timestamp'] <= 1616005200000)]

In [4]:
df = pd.concat([df_0, df_1, df_2, df_3, df_4, df_5, df_6, df_mix])
#df = pd.concat([df_0, df_1, df_2, df_3, df_4, df_5, df_6])
#df= pd.concat([df_0, df_1, df_2, df_3, df_4])

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5353 entries, 7 to 759
Data columns (total 82 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   time                                     5353 non-null   float64
 1   timestamp                                5353 non-null   int64  
 2   seconds                                  5353 non-null   float64
 3   connectivity                             5353 non-null   int64  
 4   alarmtimer:alarmtimer_fired              5353 non-null   int64  
 5   alarmtimer:alarmtimer_start              5353 non-null   int64  
 6   block:block_bio_backmerge                5353 non-null   int64  
 7   block:block_bio_remap                    5353 non-null   int64  
 8   block:block_dirty_buffer                 5353 non-null   int64  
 9   block:block_getrq                        5353 non-null   int64  
 10  block:block_touch_buffer                 5353 non

In [6]:
df = df.loc[(df['connectivity']==1)]

In [7]:
# Remove columns with constant values
df=df.loc[:, (df != df.iloc[0]).any()]

In [8]:
df.describe()

,time,timestamp,seconds,block:block_bio_backmerge,block:block_bio_remap,block:block_dirty_buffer,block:block_getrq,block:block_touch_buffer,block:block_unplug,clk:clk_set_rate,...,writeback:sb_clear_inode_writeback,writeback:wbc_writepage,writeback:writeback_dirty_inode,writeback:writeback_dirty_inode_enqueue,writeback:writeback_dirty_page,writeback:writeback_mark_inode_dirty,writeback:writeback_pages_written,writeback:writeback_single_inode,writeback:writeback_write_inode,writeback:writeback_written
count,5339.000000,5.339000e+03,5339.000000,5339.000000,5339.000000,5339.000000,5339.000000,5339.000000,5339.000000,5339.000000,...,5339.000000,5339.000000,5339.000000,5339.000000,5339.000000,5339.000000,5339.000000,5339.000000,5339.000000,5339.000000
mean,3928.291767,1.615705e+12,5.011843,1.719798,2.635138,4.167634,0.907473,4.089530,0.504402,1.585690,...,1.074171,1.165574,54.862896,0.199850,3.126054,57.984079,0.995692,0.450272,0.233564,1.406443
std,3942.898974,2.698815e+08,0.009430,2.635422,3.840664,148.127538,1.395699,156.106611,0.772641,3.435362,...,1.885175,2.269565,181.420970,0.454724,143.379991,313.307298,0.089649,0.754349,0.649198,0.668851
min,15.033600,1.615419e+12,5.008997,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,877.011500,1.615435e+12,5.010348,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,8.000000,0.000000,0.000000,9.000000,1.000000,0.000000,0.000000,1.000000
50%,1914.410000,1.615926e+12,5.010565,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,20.000000,0.000000,0.000000,21.000000,1.000000,0.000000,0.000000,1.000000
75%,6707.675000,1.615980e+12,5.011005,5.000000,7.000000,5.000000,2.000000,0.000000,1.000000,1.000000,...,1.000000,0.000000,26.000000,0.000000,0.000000,27.000000,1.000000,1.000000,0.000000,2.000000
max,13398.100000,1.616005e+12,5.285210,59.000000,72.000000,10521.000000,27.000000,11020.000000,24.000000,14.000000,...,7.000000,27.000000,11248.000000,6.000000,10448.000000,21697.000000,3.000000,8.000000,7.000000,4.000000


In [9]:
df.drop(['time','timestamp'],axis=1,inplace=True)

In [10]:
# Split dataset in training and testing
X_train, X_test = train_test_split(df, test_size=0.10, random_state=42)
X_train.shape

(4805, 72)

In [11]:
#Save selected features
feat_list=X_train.columns

In [12]:
#Feature scaling

# Min_Max_Scaler normalization
#scaler = MinMaxScaler().fit(X_train)
#scaler = QuantileTransformer(n_quantiles=1000,random_state=42).fit(X_train)
#scaler = RobustScaler().fit(X_train)
scaler= StandardScaler().fit(X_train)
# Transform training set
X_train = scaler.transform(X_train)
# Transform test set
X_test = scaler.transform(X_test)

In [66]:
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

pickle.dump(X_train, open("train_data.pkl", 'wb'))
pickle.dump(X_test, open("test_data.pkl", 'wb'))

In [13]:
# Different models

#clf = IsolationForest(random_state=42, contamination=0.2)

"""detector_list=[AutoEncoder([32,16,32], hidden_activation='relu',epochs=20,contamination=0.1),
                LOF(contamination=0.1,n_neighbors=25), COPOD(contamination=0.1),
                IForest(random_state=42,contamination=0.1), CBLOF(n_clusters=20,contamination=0.1),
                HBOS(n_bins=100,contamination=0.1), LODA(n_bins=100,n_random_cuts=100,contamination=0.1)]"""
#clf = LSCP(detector_list)
#clf = AutoEncoder([16], hidden_activation='relu',epochs=20,contamination=0.2,dropout_rate=0.0,preprocessing=True)
#clf = LOF(contamination=0.05,n_neighbors=50)
#clf = CBLOF(n_clusters=20,contamination=0.2)
#clf = VAE(encoder_neurons=[32,16],decoder_neurons=[16,32],latent_dim=2,epochs=50,contamination=0.05,verbose=1)
#clf = LODA(n_bins=100,n_random_cuts=100,contamination=0.2)
#clf = COPOD(contamination=0.2)
#clf = HBOS(n_bins=100,contamination=0.2)
#clf = SO_GAAL(contamination=0.2)
#clf = MO_GAAL(contamination=0.2)
#clf = OCSVM(kernel='rbf',gamma=0.0001, nu=0.3)


"detector_list=[AutoEncoder([32,16,32], hidden_activation='relu',epochs=20,contamination=0.1),\n                LOF(contamination=0.1,n_neighbors=25), COPOD(contamination=0.1),\n                IForest(random_state=42,contamination=0.1), CBLOF(n_clusters=20,contamination=0.1),\n                HBOS(n_bins=100,contamination=0.1), LODA(n_bins=100,n_random_cuts=100,contamination=0.1)]"

In [14]:
# Model training

#clf.fit(X_train)

In [15]:
# Model evaluation with good behaviour
#pred=clf.predict(X_test)
#unique_elements, counts_elements = np.unique(pred, return_counts=True)
#print("\t",unique_elements,"    ",counts_elements)

In [16]:
#MANUAL AUTOENCODER BUILD AND TRAIN
from sklearn.svm import OneClassSVM
autoencoder=OneClassSVM(kernel = 'rbf', gamma = 0.001, nu = 0.03).fit(X_train)

In [17]:
y_pred = autoencoder.predict(X_test)
y_pred

array([ 1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,
        1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1

In [18]:
pickle.dump(autoencoder, open("anomaly_svm.sav", 'wb'))

In [19]:
# Predict Anomalies contained in the normal behaviour
mad_outliers = np.where(y_pred == -1) 
outlier_values = df.iloc[mad_outliers]
print(outlier_values)
print(outlier_values.shape)

      seconds  block:block_bio_backmerge  block:block_bio_remap  \
9    5.025842                          5                      8   
27   5.010337                          5                      8   
63   5.009809                          0                      0   
123  5.010259                          5                      9   
137  5.010268                          0                      0   
208  5.013219                          5                      7   
216  5.010422                          0                      1   
241  5.010374                          0                      0   
286  5.010869                          5                      7   
314  5.010339                          0                      0   
382  5.010027                          0                      0   
388  5.010532                          0                      0   
476  5.010107                          0                      0   

     block:block_dirty_buffer  block:block_getrq  block:block

In [20]:
##### Attack 1: SENDING FAKE PSD VALUES ####

In [21]:
df_fakepsd=pd.read_csv("./Datasets/Rp3_3rd_capture/rp3_fakePSD.csv")

In [22]:
df_fakepsd_1 = df_fakepsd.loc[(df_fakepsd['timestamp'] >= 1615452439000) & (df_fakepsd['timestamp'] <= 1615478400000)]
df_fakepsd_2 = df_fakepsd.loc[(df_fakepsd['timestamp'] >= 1615479600000) & (df_fakepsd['timestamp'] <= 1615485600000)]

df_fakepsd = pd.concat([df_fakepsd_1, df_fakepsd_2])
df_fakepsd.drop(['time','timestamp'],axis=1,inplace=True)

In [23]:
df_fakepsd = df_fakepsd.loc[(df_fakepsd['connectivity']==1)]

In [24]:
df_fakepsd = df_fakepsd[df_fakepsd.columns.intersection(feat_list)]

In [25]:
df_fakepsd = scaler.transform(df_fakepsd)

In [26]:
#pred=clf.predict(df_fakepsd)
#unique_elements, counts_elements = np.unique(pred, return_counts=True)
#print("\t",unique_elements,"    ",counts_elements)

In [27]:
y_pred = autoencoder.predict(df_fakepsd)
mad_outliers = np.where(y_pred == -1)
outlier_values = df.iloc[mad_outliers]
print(outlier_values)
print(outlier_values.shape)

     seconds  block:block_bio_backmerge  block:block_bio_remap  \
62  5.010227                          5                      8   
76  5.010721                          5                      8   
81  5.014376                          0                      0   
92  5.010251                          0                      0   
97  5.010358                          5                      8   
..       ...                        ...                    ...   
5   5.015562                          5                      8   
6   5.011112                          5                      8   
8   5.012276                          6                     10   
10  5.010925                          0                      0   
15  5.010578                          5                      8   

    block:block_dirty_buffer  block:block_getrq  block:block_touch_buffer  \
62                         5                  3                         0   
76                         5                  3      

In [28]:
##### Attack 2: SENDING OUT PSD VALUES #### 

In [29]:
df_sendout=pd.read_csv("./Datasets/Rp3_3rd_capture/rp3_sendOut.csv")

In [30]:
df_sendout_1 = df_sendout.loc[(df_sendout['timestamp'] >= 1615795500000) & (df_sendout['timestamp'] <= 1615805280000)]
df_sendout_2 = df_sendout.loc[(df_sendout['timestamp'] >= 1615806000000) & (df_sendout['timestamp'] <= 1615809600000)]

df_sendout = pd.concat([df_sendout_1, df_sendout_2])
df_sendout.drop(['time','timestamp'],axis=1,inplace=True)

In [31]:
df_sendout = df_sendout.loc[(df_sendout['connectivity']==1)]

In [32]:
df_sendout = df_sendout[df_sendout.columns.intersection(feat_list)]

In [33]:
df_sendout = scaler.transform(df_sendout)

In [34]:
#pred=clf.predict(df_fakepsd)
#unique_elements, counts_elements = np.unique(pred, return_counts=True)
#print("\t",unique_elements,"    ",counts_elements)

In [35]:
y_pred = autoencoder.predict(df_sendout)
mad_outliers = np.where(y_pred == -1)
outlier_values = df.iloc[mad_outliers]
print(outlier_values)
print(outlier_values.shape)

       seconds  block:block_bio_backmerge  block:block_bio_remap  \
7     5.010411                          5                      8   
8     5.011067                          6                     10   
9     5.025842                          5                      8   
10    5.013875                          5                      8   
11    5.010641                          5                      8   
...        ...                        ...                    ...   
1124  5.009768                          0                      0   
1125  5.009650                          5                      7   
1126  5.010002                          0                      0   
1127  5.010446                          0                      0   
1128  5.010589                          5                      7   

      block:block_dirty_buffer  block:block_getrq  block:block_touch_buffer  \
7                            5                  3                         6   
8                        

In [36]:
##### Attack 3: WRITE #### PRIVACY LEAKAGE

In [37]:
df_write=pd.read_csv("./Datasets/Rp3_3rd_capture/rp3_write.csv")

In [38]:
df_write = df_write.loc[(df_write['timestamp'] >= 1615834554000) & (df_write['timestamp'] <= 1615843200000)]

df_write.drop(['time','timestamp'],axis=1,inplace=True)

In [39]:
df_write = df_write.loc[(df_write['connectivity']==1)]

In [40]:
df_write = df_write[df_write.columns.intersection(feat_list)]

In [41]:
df_write = scaler.transform(df_write)

In [42]:
#pred=clf.predict(df_fakepsd)
#unique_elements, counts_elements = np.unique(pred, return_counts=True)
#print("\t",unique_elements,"    ",counts_elements)

In [43]:
y_pred = autoencoder.predict(df_write)
mad_outliers = np.where(y_pred == -1)
outlier_values = df.iloc[mad_outliers]
print(outlier_values)
print(outlier_values.shape)

      seconds  block:block_bio_backmerge  block:block_bio_remap  \
7    5.010411                          5                      8   
8    5.011067                          6                     10   
9    5.025842                          5                      8   
10   5.013875                          5                      8   
11   5.010641                          5                      8   
..        ...                        ...                    ...   
705  5.011455                          0                      0   
706  5.010439                          0                      0   
710  5.010788                          0                      0   
711  5.010173                          0                      0   
714  5.010391                          0                      0   

     block:block_dirty_buffer  block:block_getrq  block:block_touch_buffer  \
7                           5                  3                         6   
8                           6          

In [44]:
##### Attack 4: ADDING RANDOM NOISE TO PSD VALUES ####

In [45]:
# Data load
df_random=pd.read_csv("./Datasets/Rp3_3rd_capture/rp3_random.csv")

In [46]:
# Remove columns with constant values
df_random = df_random.loc[(df_random['connectivity']==1)]

In [47]:
# Filter vectors according to the timestamps
df_random_new = df_random.loc[(df_random['timestamp'] >= 1615913848000) & (df_random['timestamp'] <= 1615922405000)]

df_random_new.drop(['time','timestamp'],axis=1,inplace=True)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [48]:
X_outliers6=df_random_new[df_random_new.columns.intersection(feat_list)]

In [49]:
# Transform outliers set
X_outliers6 = scaler.transform(X_outliers6)

In [50]:
#Outliers with manual autoencoder
y_pred = autoencoder.predict(X_outliers6)
mad_outliers = np.where(y_pred == -1)
outlier_values = df.iloc[mad_outliers]
print(outlier_values)
print(outlier_values.shape)

      seconds  block:block_bio_backmerge  block:block_bio_remap  \
7    5.010411                          5                      8   
9    5.025842                          5                      8   
10   5.013875                          5                      8   
15   5.009766                          5                      8   
16   5.010313                          5                      8   
..        ...                        ...                    ...   
715  5.010373                          0                      0   
717  5.010905                          5                      7   
718  5.014106                          0                      0   
720  5.009876                          0                      0   
721  5.010527                          0                      0   

     block:block_dirty_buffer  block:block_getrq  block:block_touch_buffer  \
7                           5                  3                         6   
9                           5          

In [51]:
##### Attack 5: EXCHANGE PSD VALUES OF TWO FREQUENCY BANDS ####
# Freq bands 90Mhz - [300Mhz - 1Ghz]

In [52]:
# Data load
df_exchange2=pd.read_csv("./Datasets/Rp3_3rd_capture/rp3_exchange.csv")

In [53]:
# Remove columns with constant values
df_exchange2 = df_exchange2.loc[(df_exchange2['connectivity']==1)]

In [54]:
# Filter vectors according to the timestamps
df_exchange2 = df_exchange2.loc[(df_exchange2['timestamp'] >= 1616585145000) & (df_exchange2['timestamp'] <= 1616589463000)]

df_exchange2.drop(['time','timestamp'],axis=1,inplace=True)

In [55]:
X_outliers9=df_exchange2[df_exchange2.columns.intersection(feat_list)]

In [56]:
# Transform outliers set
X_outliers9 = scaler.transform(X_outliers9)

In [57]:
#Outliers with manual autoencoder
y_pred = autoencoder.predict(X_outliers9)
mad_outliers = np.where(y_pred == -1)
outlier_values = df.iloc[mad_outliers]
print(outlier_values)
print(outlier_values.shape)

      seconds  block:block_bio_backmerge  block:block_bio_remap  \
8    5.011067                          6                     10   
9    5.025842                          5                      8   
13   5.010144                          5                      8   
17   5.010375                          5                      8   
19   5.010187                          6                     13   
20   5.010433                          5                      8   
22   5.011108                          5                      7   
27   5.010337                          5                      8   
30   5.009892                          6                     13   
35   5.011008                          0                      0   
52   5.010378                          5                      8   
56   5.010145                          5                      8   
60   5.010232                          6                     13   
71   5.010436                          0                      

In [58]:
##### Attack 6: Hide a transmission ####
# 200Mhz - 250Mz

In [59]:
# Data load
df_hide=pd.read_csv("./Datasets/Rp3_3rd_capture/rp3_hide.csv")

In [60]:
# Remove columns with constant values
df_hide = df_hide.loc[(df_hide['connectivity']==1)]

In [61]:
# Filter vectors according to the timestamps
df_hide = df_hide.loc[(df_hide['timestamp'] >= 1616672277000) & (df_hide['timestamp'] <= 1616676574000)]

df_hide.drop(['time','timestamp'],axis=1,inplace=True)

In [62]:
X_outliers10 = df_hide[df_hide.columns.intersection(feat_list)]

In [63]:
# Transform outliers set
X_outliers10 = scaler.transform(X_outliers10)

In [64]:
#Outliers with manual autoencoder
y_pred = autoencoder.predict(X_outliers10)
mad_outliers = np.where(y_pred == -1)
outlier_values = df.iloc[mad_outliers]
print(outlier_values)
print(outlier_values.shape)

      seconds  block:block_bio_backmerge  block:block_bio_remap  \
12   5.009667                          5                      8   
19   5.010187                          6                     13   
46   5.010555                          5                      8   
69   5.010643                          0                      0   
94   5.017679                          0                      2   
181  5.010274                          0                      0   
233  5.010306                          0                      0   
272  5.010543                          0                      0   
279  5.010393                          0                      1   
303  5.009771                          0                      2   
313  5.010464                          0                      1   

     block:block_dirty_buffer  block:block_getrq  block:block_touch_buffer  \
12                          5                  3                         0   
19                          5          